# Threads

In diesem Notebook schauen wir uns Threads an. Nachdem wir das grundlegende Prinzip illustriert haben, schauen wir uns zwei Beispiele an: Das erste Beispiel zeigt, wie wir dank Threads Berechnungen parallel durchführen können. Mit dem zweiten Beispiel illustrieren wir die Probleme, die beim arbeiten mit Threads auftreten können. 

### Vorbereitung

Wenn wir mit mehreren Threads arbeiten, ist die Ausgabe auf die Konsole meistens sehr unübersichtlich, da die Reihenfolge in der auf die Konsole geschrieben wird nicht deterministisch ist. In Jupyter notebooks ist es sogar gänzlich unmöglich, aus mehreren Threads gleichzeitig Output zu generieren. Deshalb führen wir als erstes eine Logging Klasse ein. Diese verwaltet einzelne Logging-Nachrichten mittels einer LinkedList. Um die Übersicht zu behalten, wird jede Nachricht mit einem Datum versehen. 

In [ ]:
class LogMessage {
    Date date;
    String message;
    LogMessage next;
    
    public LogMessage (String message) {
        this.date = new Date();
        this.message = message;
        this.next = null;
    }
    
    public String toString() { 
        java.text.SimpleDateFormat sdfDate = 
            new java.text.SimpleDateFormat("yyyy-MM-dd HH:mm:ss:S");
        return sdfDate.format(this.date) +" : " + message;
    }
}

class Logger {
    LogMessage head = null;
    LogMessage tail = null;
    
    synchronized void log(String message ) {
        LogMessage newMessage = new LogMessage(message);
        if (head == null) {
            this.head = newMessage;
            this.tail = head;
        } else {
            this.tail.next = newMessage;
            this.tail = newMessage;
        }
    }
    
    void print() {
        LogMessage m = head;
        while (m != null) {
            System.out.println(m);
            m = m.next;
        }
    }
    
}

Um den logger zu nutzen, instanzieren wir ein neues Objekt, und loggen unsere Ausgaben jeweils durch Aufruf der ```log``` Methode.  Alle Logeinträge können dann mit ```print``` angezeigt werden. 

In [ ]:
Logger logger = new Logger();
logger.log("Ausgabe 1");
logger.log("Ausgabe 2");
logger.print()


### Threads erzeugen

Als erstes illustrieren wir den grundlegenden Mechansimus um Threads zu erzeugen.
Um einen neuen Thread zu erzeugen, müssen wir als erstes eine Klasse schreiben, die von der Klasse  ```Thread``` erbt, und die run Methode überschreibt. Damit wir die Ausgabe sehen, übergeben wir noch einen Namen für den Thread sowie ein ```Logger``` Objekt.

In [ ]:
class MyThread extends Thread {

    private Logger logger;
    private String name;

    MyThread(String name, Logger logger) {
        this.logger = logger;
        this.name = name;
    }
    
    public void run() { 
        java.util.Random rng = new java.util.Random();
        
        for (int i = 0; i < 10; i++) {
            logger.log("Hello from thread " +name);
            try {
                Thread.sleep(rng.nextInt(1000));
            } catch (InterruptedException e) {
                logger.log("got interruped");
            }
        }
    }
}

Um dann den eigentlichen Thread zu starten, müssen wir eine Instanz unserer Thread-Klasse erstellen und die start Methode aufrufen.

In [ ]:
Logger logger = new Logger();
MyThread thread1 = new MyThread("Thread 1", logger);
MyThread thread2 = new MyThread("Thread 2", logger);
thread1.start();
thread2.start();

In [ ]:
logger.print();

#### Miniübung:
* Führen Sie statt ```start``` die Methode ```run``` direkt aus. Was passiert? Weshalb?

### Beispiel 1: Numerische Integration

Als erstes nützliches Beispiel für die Anwendung von Threads schreiben wir ein Programm, welches eine gegebene Funktion numerisch, mithilfe der Quadratregel,  integriert. 

Um Funktionen zu implementieren schreiben wir uns ein Interface ```Function```:

In [ ]:

interface Function {
    double apply(double x);
}

#### Miniübung: 
* Implementieren Sie Klassen für verschiedene Funktionen, wie zum Beispiel $sin, cos, x^2, x^3$, welche das Interface implementieren. 

Als nächstens implementieren wir den Integrator. Dieser nimmt ein Interval, gegeben durch zwei Punkte $x_1$ und $x_2$ sowie ein Element $dx$, welches die Grösse von jedem Rechteck angibt. Die Methode ```run``` führt dann die eigentliche Berechnung aus. Die Methode ```getValue``` gibt den berechneten Wert zurück.

In [ ]:
class Integrator {
    
    double value = 0; 
    double x1;
    double x2; 
    double dx;
    Function f;
    
    public Integrator(double x1, double x2, double dx, Function f) {
        this.x1 = x1;
        this.x2 = x2;
        this.dx = dx;
        this.f = f;
    }
    
    public void run() {
        int N = (int) ((x2 - x1) / dx);
        double s = 0;

        for (int i = 1; i <= N; i++) {
            s += f.apply(x1 + i * dx) * dx;
        }
        value = s;
    }
    public double getValue() { return value; }
}


Wir können nun das Integral einer Funktion wie folgt berechnen:

In [ ]:
class SineFunction implements Function {
    public double apply(double d) { return Math.sin(d); }
}

Integrator integrator = new Integrator(-Math.PI, Math.PI, 1e-8, new SineFunction());
integrator.run();
integrator.getValue();

#### Übung:

* Passen Sie die Klasse ```Integrator``` so an, dass diese von ```Thread``` erbt.
* Starten Sie den Integrator nun mit der ```start``` Methode (die von ```Thread``` geerbt wurde). Was beobachten Sie?
* Schreiben Sie eine neue Methode ```integrate``` (ausserhalb der Klasse Integrator), welche zusätzlich zu $x1, x2, dx$ und $f$ noch die anzahl Threads nimmt, die erzeugt werden können und die Berechnung dann auf mehrere Threads aufteilt. 
    * Hinweis: Um zu warten bis ein Thread fertig ist können Sie die Methode ```join``` der Thread Klasse nutzen. 

### Beispiel 2: Ein einfacher Zähler

In unserem zweiten Beispiel illustrieren wir, was beim gleichzeitigen Zugriff auf Variablen schiefgehen kann. 
Dazu schreiben wir eine Klasse ```Counter``` welche einen einfachen Zähler implementiert.

In [ ]:
public class Counter {

    private int c = 0;

    public void increment() {
        c++;
    }

    public void decrement() {
        c--;
    }

    public int value() {
        return c;
    }
}

Wir implementieren nun eine Klasse, welche jeweils den Zähler inkrementiert und wieder dekrementiert. Am Ende wird dann der Wert des Zählers ausgegeben. 

In [ ]:
class MyThread extends Thread {

    String name;
    Counter counter;
    Logger logger;
    
    public MyThread(String name, Counter counter, Logger logger) {
        this.name = name;
        this.counter = counter;
        this.logger = logger;
    }
    
    public void run() {
        
        java.util.Random rng = new java.util.Random();
        
        for (int i = 0; i < 100; i++) {         
            counter.increment();                     
            counter.decrement();
            
            logger.log(name +" "+counter.value());       

            try {
                Thread.sleep(rng.nextInt(10));
            } catch (InterruptedException e) {
                logger.log("got interruped");
            }
        }
        
    }
}

Wir starten nun das Programm und sehen uns dann den Output an. 

In [ ]:
Logger logger = new Logger();
Counter counter = new Counter();

for (int i = 0; i < 10; i++) {
    MyThread thread = new MyThread("Thread " + i, counter, logger);
    thread.start();
}

Wir würden erwarten, dass der Wert der ausgegeben wird immer 0 ist. Zu unserer Überraschung sehen wir, dass dem nicht so ist. 

In [ ]:
logger.print()

#### Miniübung

* Nutzen Sie ```synchronize``` um das Programm so zu synchronisieren, dass es die erwartete Ausgabe liefert.